In [ ]:
import numpy as np
import pandas as pd
import random
from random import *
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.optimizers import RMSprop
from collections import deque
from tqdm import tqdm
import keras

In [ ]:
types = ['6633', '6024', '1062', '6310', '6128', '6287', '1075', '1076',
       '6756', '6757', '1101', '6494', '6662', '6705', '6549', '6552',
       '6527', '6531', '1099', '1104', '6754', '6575', '6706', '6589',
       '6232', '6752', '6574', '6053', 'P828', '6692', '6721', '6663',
       '6734', '6681', '6682', '6561', '6750', '6728', '6648', '6584',
       '6102', '6630', '6406', '6407', '1106', '1107', '6542', '6723',
       '6356', '6700', '6572', '6573', '6218', '1017', 'P130', '6763',
       '6650', '6653', '6454', '6452', '6696', '8L39', '8L72', '6698',
       '6433', '6629', '6760', '6380', 'P099', '6585', '6581', '6447',
       '6689', '6761', '6040', '6515', '6217', '6747', '6769', '6738',
       '6751', '6112', '6753', '6301', '6271', '6432', '6675', '6470',
       '6647', '1018', '6238', '6324', '6733', '6206', 'P163', '6719',
       '6205', '6660', '6453', '6655', '6646', '6244', '6695', '6539',
       '6736', '6683', '6634', '6475', 'P147', '6642', '6684', '6477',
       '6608', '6708', 'P837', 'P820', 'P835', '8K52']
machine = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
machine_state = [[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]]

In [ ]:
new_df = pd.read_csv("C:\\eric\\Projects\\APS_Betha\\Codes\\datasets\\Sch\\data_scheduling.csv")
#           ORDER_ID type  process_time  due_date      
# 0    20171221-2257  6633            10        10      
# 1    20171221-1374  6024             6        10      
# 2    20171221-1348  1062             3        10      
# 3    20171221-1315  6310             2        10      
# 4    20171221-1313  6310             5        10 

len(new_df['type'].unique())

new_table = pd.read_csv("C:\\eric\\Projects\\APS_Betha\\Codes\\datasets\\Sch\\data_const.csv")
#      machine  type
# 0      12216  1075
# 1      12216  1076
# 2      12216  1099
# 3      12216  1101
# 4      12216  1104

#------------------------------------------------------------------------------------------------#
# new_table01 = 새로운 const df 
new_table01 = new_table.groupby('machine').agg(lambda x: ','.join(set(x)))
new_table01 = new_table01.reset_index()
new_table01 = new_table01.reset_index()
del new_table01['machine']
new_table01.columns = ['machine','type']
new_table01
for i in range(0,16):
    new_table01['type'][i] = new_table01['type'][i].split(',')

In [ ]:
import keras

class CustomHistory(keras.callbacks.Callback):
    def __init__(self):
        self.train_loss = []
        self.val_loss = []
        self.train_acc = []
        self.val_acc = []
        
    def on_epoch_end(self, batch, logs={}):
        self.train_loss.append(logs.get('loss'))
        self.val_loss.append(logs.get('val_loss'))
        self.train_acc.append(logs.get('acc'))
        self.val_acc.append(logs.get('val_acc'))

In [ ]:
import random

custom_hist = CustomHistory()
custom_hist.__init__()

class DQN:
    def __init__(self):
        self.memory = deque(maxlen=2000)
        
        self.gamma = 0.85
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.005
        self.tau = .125
        
        self.model = self.create_model()
        self.target_model = self.create_model()
        
    def create_model(self):
        model = Sequential()
        model.add(Dense(64, input_dim=198, activation="relu"))
        model.add(Dense(32, activation="relu"))
        model.add(Dense(16, activation="relu"))
        model.add(Dense(118))
        model.compile(loss="mean_squared_error", optimizer=Adam(lr=self.learning_rate))
        
        return model
    
    def act(self, cur_state):
        self.epsilon *= self.epsilon_decay
        self.epsilon = max(self.epsilon_min, self.epsilon)
        random_val = np.random.random()

        if random_val < self.epsilon:
            blank = []
            for i in range(len(new_table01['type'])):
                for j in range(len(new_table01['type'][i])):
                    if new_table01['type'][i][j] == c_type:
                        blank.append(i)
            return random.choice(blank)

        else:
            blank = []
            for i in range(len(new_table01['type'])):
                for j in range(len(new_table01['type'][i])):
                    if new_table01['type'][i][j] == c_type:
                        blank.append(i)
            score = self.model.predict(state)

            blank2 = []
            for i in blank:
                ax = score[0][i]
                blank2.append(ax)
            max_q_val = max(blank2)

            index_of_max = np.where(score[0] == max_q_val)
            iom = index_of_max[0][0]
            # np.argmax(self.model.predict(state)[0])
            return iom

    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append([state, action, reward, next_state, done])
        
    def replay(self):
        batch_size = 36
        if len(self.memory) < batch_size:
            return
        
        samples = random.sample(self.memory, batch_size)
        X=[]
        Y=[]
        for sample in samples:
            state, action, reward, next_state, done = sample
            target = self.target_model.predict(state)
            if done:
                target[0][action] = reward
            else:
                Q_future = max(self.target_model.predict(next_state)[0])
                target[0][action] = reward + Q_future * self.gamma
            X.append(state)
            Y.append(target)
        X = np.array(X)
        Y = np.array(Y)    
        self.model.fit(X, Y, epochs=1, batch_size=64 ,verbose=0, callbacks=[custom_hist])

    def target_train(self):
        weights = self.model.get_weights()
        target_weights = self.target_model.get_weights()
        for i in range(len(target_weights)):
            target_weights[i] = weights[i]* self.tau + target_weights[i]*(1-self.tau)
        self.target_model.set_weights(target_weights)
        
    def save_model(self, fn):
        self.model.save(fn)

In [ ]:
dqn_agent = DQN()

job_change_count = 0
validation_date_count = 0
num_games = 500
done = False


for game in tqdm(range( 0, num_games )):
    
    done = False

    current = np.array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.]])
    current = current.flatten()
    current = current.tolist()

    machine_state_np = np.array(machine_state)
    machine_state_flatten = machine_state_np.flatten()
    # machine_state
    machine_state_final = machine_state_flatten.tolist()
    # next machine_state
    state = current + machine_state_final
    state = np.array(state)
    state = state.reshape(1,198)

    reward_total =0 
    validation_date_count = 0
    job_change_count = 0

    action_list = [] 
    for j in range(1,len(new_df)): # order list
        #state, action, reward, next_state, done 
        if j == len(new_df)-2:
            done = True

        # order check 
        c_type = new_df['type'][j]  
        c_process_time = new_df['process_time'][j]
        c_due = new_df['due_date'][j]

        #c_type encoding 
        current = np.identity(118)[types.index(c_type):types.index(c_type)+1]
        current = current.flatten()
        current = current.tolist()

        #action
        action = dqn_agent.act(state)
        #print(action) 
        action_list.append(action)  

        # reward
        if len(machine[action]) == 0 or machine[action][-1] == c_type:
            job_change = 0
        else:
            job_change = 1
            job_change_count += 1


        if machine_state[action][4] < 0:
            validation_due_date = machine_state[action][4]
            validation_due_date = -1 * validation_due_date
            validation_date_count += validation_due_date
        else:
            validation_due_date = 0
        

        #if machine_state[action][4] < 0:
        #    validation_due_date = machine_state[action][4]
        #    validation_date_count += 1
        #else:
        #    validation_due_date = 0

        reward = 50000 + (-1*job_change) + (-5*validation_due_date)
        reward_total += reward


        #env changed 
        ####
        for j in range(c_process_time):
            machine[action].append(c_type)


        #status update

        # 0 #status of machine #processed slots 
        for k in range(0, len(machine)): 
            machine_len = len(machine[k])
            machine_state[k][0] = machine_len

        # 1 #status of order #number of order 
        for k in range(0, len(machine)):
            machine_state[k][1] = c_process_time 

        # 2 #status of order #due date of order
        for k in range(0, len(machine)):
            machine_state[k][2] = c_due
        
        # 3 #compare between state and order # same block indicator 
        for k in range(0, len(machine)):
            if len(machine[k]) ==0 or machine[k][-1] == c_type:
                machine_state[k][3] = 1
            else:
                machine_state[k][3] = 0    

        # 4 #compare between state and order # due date violation 
        for k in range(0, len(machine)):
            machine_state[k][4] = c_due - (len(machine[k]) + c_process_time) 
        
        #print(machine_state)



        machine_state_np = np.array(machine_state)
        machine_state_flatten = machine_state_np.flatten()
        # machine_state
        machine_state_final = machine_state_flatten.tolist()
        # next machine_state
        next_state = current + machine_state_final
        next_state = np.array(next_state)
        next_state = next_state.reshape(1,198)
        ##################################################################
        #state, action, reward, next_state, done 저장 및 학습


        dqn_agent.remember(state, action, reward, next_state, done)

        #dqn_agent.replay()
        #dqn_agent.target_train()

        state = next_state

        if done:
            break
        #dqn_agent.save_model("success.model") # 여기도,, 적어도 episode 몇백번은 반복을 하고 나서 model save 를 해야지.. 
        #print(machine)
        #print(c_type)
    dqn_agent.replay()
    dqn_agent.target_train()
    dqn_agent.save_model("success.model") # 여기도,, 적어도 episode 몇백번은 반복을 하고 나서 model save 를 해야지.. 
    
    print("job_change_count = " + str(job_change_count) + ", violation_date_count = " + str(validation_date_count))    
    print(reward_total)
    print(action_list)

In [ ]:
#%matplotlib inline
import matplotlib.pyplot as plt


print(custom_hist)
fig, loss_ax = plt.subplots()

acc_ax = loss_ax.twinx()

loss_ax.plot(custom_hist.train_loss, 'y', label='train loss')
loss_ax.plot(custom_hist.val_loss, 'r', label='val loss')

acc_ax.plot(custom_hist.train_acc, 'b', label='train acc')
acc_ax.plot(custom_hist.val_acc, 'g', label='val acc')

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuray')

loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')

plt.show()

In [ ]:
dqn_agent.save_model("model.h5")